In [2]:
import sys, os, time
#os.environ["CUDA_VISIBLE_DEVICES"]="0"
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as distrib
import torch.multiprocessing as mp
from torch.utils.data import Dataset, DataLoader
import gym
import numpy as np
%matplotlib notebook
#%matplotlib tk
import matplotlib.pyplot as plt
#plt.switch_backend('Qt5Agg') #('Qt5Agg')
import foundation as fd
from foundation import models
from foundation import util
from foundation import train

from langimg import *

np.set_printoptions(linewidth=120)

/is/ei/fleeb/workspace/foundation


In [33]:
x = torch.zeros(5,3, requires_grad=True)

[autoreload of langimg failed: Traceback (most recent call last):
  File "/is/ei/fleeb/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/is/ei/fleeb/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 450, in superreload
    update_generic(old_obj, new_obj)
  File "/is/ei/fleeb/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 387, in update_generic
    update(a, b)
  File "/is/ei/fleeb/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 342, in update_class
    if update_generic(old_obj, new_obj): continue
  File "/is/ei/fleeb/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 387, in update_generic
    update(a, b)
  File "/is/ei/fleeb/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 266, in update_function
    setattr(old, name, getattr(new, name))
ValueE

In [34]:
x

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], requires_grad=True)

In [36]:
dists = F.pdist(x).pow(2)
dists

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], grad_fn=<PowBackward0>)

In [37]:
f = 1 / dists[dists > 0]
f

tensor([], grad_fn=<MulBackward0>)

In [38]:
f.mean()

tensor(nan, grad_fn=<MeanBackward0>)

In [18]:
mx = lt.max(-1)[0]
sel = mx > 1.

In [19]:
sel

tensor([ True, False,  True,  True,  True])

In [20]:
lt[sel, :] /= mx[sel].unsqueeze(-1)

In [32]:
lt

tensor([[-1.4462,  0.2269,  1.0000],
        [-0.0323, -1.2539, -1.1374],
        [ 1.0000, -0.7698, -1.0279],
        [ 0.1622, -0.2820, -0.2243],
        [ 1.0000,  0.3112,  0.5726]])

In [7]:


mn = lt.min(-1)[0]
sel = mn < -1
lt[sel, :] /= mn[sel].abs()

RuntimeError: The size of tensor a (3) must match the size of tensor b (2) at non-singleton dimension 1

In [2]:
args = util.NS()

args.device = 'cuda:0'
args.seed = 0

args.logdate = True
args.tblog = True
args.txtlog = False
args.saveroot = '../trained_nets'

args.dataset = 'mnist'
args.indexed = True
args.use_val = False

args.num_workers = 4
args.batch_size = 128

args.start_epoch = 0
args.epochs = 10

args.optim_type = 'adam'
args.lr = 1e-2
args.weight_decay = 1e-4
args.momentum = 0


args.latent_dim = 20

args.zero_embedding = True
args.normalize_latent = False
args.latent_lr = None

args.name = 'direct_decoder_dim{}'.format(args.latent_dim)

args.save_model = False


now = time.strftime("%y-%m-%d-%H%M%S")
if args.logdate:
    args.name = os.path.join(args.name, now)
args.save_dir = os.path.join(args.saveroot, args.name)
print('Save dir: {}'.format(args.save_dir))

if args.tblog or args.txtlog:
    util.create_dir(args.save_dir)
    print('Logging in {}'.format(args.save_dir))
logger = util.Logger(args.save_dir, tensorboard=args.tblog, txt=args.txtlog)

# Set seed
if not hasattr(args, 'seed') or args.seed is None:
    args.seed = util.get_random_seed()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
try:
    torch.cuda.manual_seed(args.seed)
except:
    pass

if not torch.cuda.is_available():
    args.device = 'cpu'
print('Using device {} - random seed set to {}'.format(args.device, args.seed))

Save dir: ../trained_nets/direct_decoder_dim20/19-10-01-155147
Logging in ../trained_nets/direct_decoder_dim20/19-10-01-155147
Using device cuda:0 - random seed set to 0


In [3]:
datasets = train.load_data(args=args)

loaders = train.get_loader(*datasets, batch_size=args.batch_size, num_workers=args.num_workers, 
                           shuffle=True, drop_last=False,)

trainloader, testloader = loaders[0], loaders[-1]
valloader = None if len(loaders) == 2 else loaders[1]

print('traindata len={}, trainloader len={}'.format(len(datasets[0]), len(trainloader)))
if valloader is not None:
    print('valdata len={}, valloader len={}'.format(len(datasets[1]), len(valloader)))
print('testdata len={}, testloader len={}'.format(len(datasets[-1]), len(testloader)))
print('Batch size: {} samples'.format(args.batch_size))

traindata len=60000, trainloader len=469
testdata len=10000, testloader len=79
Batch size: 128 samples


In [4]:
# Define Model
args.total_samples = {'train': 0, 'test': 0}
epoch = 0
best_loss = None
all_train_stats = []
all_test_stats = []
    
model = DirectDecoder(latent_dim=args.latent_dim, out_shape=(1, 28, 28), vocab_size=len(datasets[0]),
                      
                      normalize_latent=args.normalize_latent, zero_embedding=args.zero_embedding,

                          nonlin='prelu', output_nonlin='sigmoid',
                          channels=[32, 16, 8], kernels=[3, 3, 3], ups=[2, 2, 2], upsampling='bilinear',
                          output_norm_type=None,
                          hidden_fc=[64, 128],
                          )

# optim = 

model.set_optim(optim_type=args.optim_type, lr=args.lr, weight_decay=args.weight_decay, momentum=args.momentum)
# optim = util.get_optimizer('sgd', model.parameters(), )
scheduler = None#torch.optim.lr_scheduler.StepLR(optim, step_size=6, gamma=0.2)

model.to(args.device)
print(model)
print('Model has {} parameters'.format(util.count_parameters(model)))

DirectDecoder(
  (table): Embedding(60000, 20)
  (dec): Decoder(
    (fc): Sequential(
      (0): Linear(in_features=20, out_features=64, bias=True)
      (1): PReLU(num_parameters=1)
      (2): Linear(in_features=64, out_features=128, bias=True)
      (3): PReLU(num_parameters=1)
      (4): Linear(in_features=128, out_features=512, bias=True)
      (5): PReLU(num_parameters=1)
    )
    (deconv): Sequential(
      (0): DeconvLayer(
        (deconv): Sequential(
          (0): Upsample(size=(7, 7), mode=bilinear)
          (1): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (norm): InstanceNorm2d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (nonlin): PReLU(num_parameters=1)
      )
      (1): DeconvLayer(
        (deconv): Sequential(
          (0): Upsample(size=(14, 14), mode=bilinear)
          (1): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (norm): InstanceNorm2d(8, eps=1e-

In [5]:
lr = model.optim.param_groups[0]['lr']
for _ in range(10):
    
    old_lr = lr
    if scheduler is not None:
        scheduler.step()
    lr = model.optim.param_groups[0]['lr']
    
    if lr != old_lr:
        print('--- lr update: {:.3E} -> {:.3E} ---'.format(old_lr, lr))
    
    train_stats = util.StatsMeter('lr', tau=0.1)
    train_stats.update('lr', lr)

    train_stats = train.run_epoch(model, trainloader, args, mode='train',  
                                      epoch=epoch, print_freq=10, logger=logger, silent=True,
                                      viz_criterion_args=['reconstruction', 'original'],
                                      stats=train_stats, )
    
    all_train_stats.append(train_stats)
    
    print('[ {} ] Epoch {} Train={:.3f} ({:.3f})'.format(
        time.strftime("%H:%M:%S"), epoch+1,
        train_stats['loss-viz'].avg.item(), train_stats['loss'].avg.item(),
    ))
    
    if args.save_model:
        
        av_loss = test_stats['loss'].avg.item()
        is_best = best_loss is None or av_loss < best_loss
        if is_best:
            best_loss = av_loss
        
        path = train.save_checkpoint({
            'epoch': epoch,
            'args': args,
            'model_str': str(model),
            'model_state': model.state_dict(),
            'all_train_stats': all_train_stats,
            'all_test_stats': all_test_stats,
            
        }, args.save_dir, is_best=is_best, epoch=epoch+1)
        print('--- checkpoint saved to {} ---'.format(path))

    epoch += 1

print('Done')

/is/ei/fleeb/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:2479: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/is/ei/fleeb/workspace/foundation/foundation/util/stats.py:180: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.val = torch.tensor(val).float()


[ 15:51:57 ] Epoch 1 Train=0.151 (0.517)
[ 15:52:01 ] Epoch 2 Train=0.087 (0.356)
[ 15:52:04 ] Epoch 3 Train=0.074 (0.308)
[ 15:52:08 ] Epoch 4 Train=0.070 (0.290)
[ 15:52:11 ] Epoch 5 Train=0.069 (0.281)
[ 15:52:15 ] Epoch 6 Train=0.069 (0.276)
[ 15:52:19 ] Epoch 7 Train=0.068 (0.273)
[ 15:52:23 ] Epoch 8 Train=0.068 (0.272)
[ 15:52:27 ] Epoch 9 Train=0.068 (0.270)
[ 15:52:31 ] Epoch 10 Train=0.068 (0.270)
Done
